# Data
Download the IMDB reviews if its not already in the local cache.
Load from the local cache and split to train and test sets.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import os
import ssl
from tensorflow.keras.utils import get_file
try:
    from notebooks.data import load_imdb
except ModuleNotFoundError:
    from data import load_imdb

if (not os.environ.get('PYTHONHTTPSVERIFY', '')
        and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

zip_file = get_file('aclImdb.zip', origin='http://mng.bz/0tIo', extract=True)
imdb_dir = zip_file.replace('.zip', '')
(train_texts, train_labels), (test_texts, test_labels) = load_imdb(imdb_dir)

print('An example from the reviews:')
print('============================')
print(f'Text : {train_texts[0][:200]}...')
print(f'Label: {train_labels[0]}')

An example from the reviews:
Text : Entertainment Tonight has been going down hill for the last few years, but as of last night (Aug 17th 2006) they reached a new low.<br /><br />In an effort to try to hype up their broadcast, they deci...
Label: neg


# Prepare Data
Initialize a `CharVectorizer`. The vectorizer will be fitted on the 
`train_texts`. We won't use any word tokenizer, so the words will be splitted
on spaces. The vocabulary will have only the characters set on the 
`characters` attribute, the PAD and the OOV token.

In [3]:
from keras_nlp import CharVectorizer

char_vectorizer = CharVectorizer(
    characters='abcdefghijklmnopqrstuvwxyz', oov_token='#')
char_vectorizer.fit_on_texts(train_texts)
# The vectorizer's number of tokens: num_chars + PAD + OOV token
print(f'Vectorizer number of tokens: {len(char_vectorizer.token2id)}')

2020-Apr-09 23:02:28 [INFO    :CharVectorizer] - Creating vocabulary.


25000/25000 [==============================] - 3s 132us/step
Vectorizer number of tokens: 28


## Vectorize Data
We apply the fitted vectorizer onto the train and test texts. 
We keep `max_tokens` per text and `max_characters` per token.

In [4]:
max_tokens, max_characters = 1000, 10
X_train = char_vectorizer.texts_to_vectors(
    train_texts, shape=(max_tokens, max_characters))

2020-Apr-09 23:02:35 [INFO    :CharVectorizer] - Converting texts to vectors.


25000/25000 [==============================] - 11s 443us/step


2020-Apr-09 23:02:47 [INFO    :CharVectorizer] - Reshaping vectors to shape (1000, 10).


25000/25000 [==============================] - 8s 304us/step


**Attention**: We *must* pass the same shape `(max_tokens, max_characters)` 
when converting different text sets. If we don't, then it is almost certain 
that the results will have different numbers of columns because the sets are 
likely to have different number of tokens.

In [5]:
X_test = char_vectorizer.texts_to_vectors(
    test_texts, shape=(max_tokens, max_characters))

2020-Apr-09 23:02:59 [INFO    :CharVectorizer] - Converting texts to vectors.


25000/25000 [==============================] - 11s 450us/step


2020-Apr-09 23:03:13 [INFO    :CharVectorizer] - Reshaping vectors to shape (1000, 10).


25000/25000 [==============================] - 8s 312us/step


## Decode Data
We print a decoded fragment of a encoded text as an example.

In [6]:
decoded = char_vectorizer.vectors_to_texts(X_test[:1])
print('Example fragment of decoded review')
print('==================================')
print(decoded[0][67:76])

2020-Apr-09 23:03:23 [INFO    :CharVectorizer] - Converting vectors to texts.


1/1 [==============================] - 0s 8ms/step
Example fragment of decoded review
[['h', 'e'], ['s', 'h', 'e'], ['i', 's'], ['w', 'r', 'o', 'n', 'g'], ['i'], ['w', 'a', 's'], ['r', 'a', 't', 'h', 'e', 'r'], ['p', 'u', 'z', 'z', 'l', 'e', 'd'], ['b', 'y']]


## Encode Labels
The labels are strings ('pos' / 'neg'). We will convert them to numbers.

In [7]:
from sklearn.preprocessing import label_binarize

y_train = label_binarize(train_labels, classes=['neg', 'pos'])
y_test = label_binarize(test_labels, classes=['neg', 'pos'])

## Keep a Validation Set

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, stratify=y_train, shuffle=True, random_state=44)

print(f'X_train: {X_train.shape}')
print(f'X_dev  : {X_val.shape}')
print(f'X_test : {X_test.shape}')

X_train: (18750, 1000, 10)
X_dev  : (6250, 1000, 10)
X_test : (25000, 1000, 10)


# Neural Network
Here we define a toy network for demonstration purpose.

In [9]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, TimeDistributed, \
    Dropout, Flatten, Dense

chars_input = Input(shape=(
    max_tokens,
    max_characters,
),
                    name='Input',
                    dtype='int32')
chars_embeddings = TimeDistributed(Embedding(
    input_dim=char_vectorizer.num_tokens,
    output_dim=20,
    input_length=max_characters,
    mask_zero=False,
    trainable=True),
                                   name='Embeddings')(chars_input)
x = Dropout(0.4, name='Input_Dropout')(chars_embeddings)
x = Flatten(name='Flatten')(x)
x = Dropout(0.4, name='Dropout')(x)
predictions = Dense(1, activation='sigmoid', name='Predictions')(x)
model = Model(chars_input, predictions)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 1000, 10)]        0         
_________________________________________________________________
Embeddings (TimeDistributed) (None, 1000, 10, 20)      560       
_________________________________________________________________
Input_Dropout (Dropout)      (None, 1000, 10, 20)      0         
_________________________________________________________________
Flatten (Flatten)            (None, 200000)            0         
_________________________________________________________________
Dropout (Dropout)            (None, 200000)            0         
_________________________________________________________________
Predictions (Dense)          (None, 1)                 200001    
Total params: 200,561
Trainable params: 200,561
Non-trainable params: 0
_______________________________________________________

## Train Model

In [10]:
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(
    X_train, y_train, batch_size=50, epochs=5, validation_data=(X_val, y_val))

Train on 18750 samples, validate on 6250 samples
Epoch 1/5
18750/18750 [==============================] - 20s 1ms/sample - loss: 0.6947 - acc: 0.5464 - val_loss: 0.6700 - val_acc: 0.5893
Epoch 2/5
18750/18750 [==============================] - 16s 875us/sample - loss: 0.5997 - acc: 0.6810 - val_loss: 0.6528 - val_acc: 0.6202
Epoch 3/5
18750/18750 [==============================] - 17s 903us/sample - loss: 0.5096 - acc: 0.7409 - val_loss: 0.6609 - val_acc: 0.6328
Epoch 4/5
18750/18750 [==============================] - 17s 919us/sample - loss: 0.4538 - acc: 0.7789 - val_loss: 0.6833 - val_acc: 0.6230
Epoch 5/5
18750/18750 [==============================] - 16s 878us/sample - loss: 0.4204 - acc: 0.7988 - val_loss: 0.7063 - val_acc: 0.6229


## Model Evaluation

In [ ]:
scores = model.evaluate(X_test, y_test)
print(f'Evaluation accuracy: {100*scores[1]:.2f}%')

## Plot Accuracy and Loss

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

f, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(12, 6))
ax1.plot(history.epoch, history.history['acc'], label='Training')
ax1.plot(history.epoch, history.history['val_acc'], label='Validation')
ax1.set_title('Accuracy')

ax2.plot(history.epoch, history.history['loss'], label='Training')
ax2.plot(history.epoch, history.history['val_loss'], label='Validation')
ax2.set_title('Loss')
ax2.set_xlabel('epochs')
ax2.legend()

plt.show()